Imports

In [1]:
import numpy as np
import pandas as pd
import sklearn as sklearn
from sklearn.ensemble import HistGradientBoostingClassifier
print('Now using sklearn version '+sklearn.__version__)
print('Packages are Ready!')
print("Hola")

Now using sklearn version 1.0.2
Packages are Ready!
Hola


Loading data

In [23]:
print('Loading data...')
"""we are also importing training and testing data from competition because they might become handy"""

test=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/Santander-Transaction-Competition/test.csv')
train=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/Santander-Transaction-Competition/train.csv')

"""reading data data"""

x_test=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/Santander-Transaction-Competition/x_test.csv')
x_train=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/Santander-Transaction-Competition/x_train.csv')

print('Data is ready!!!!')


Loading data...
Data is ready!!!!


In [25]:
x_train

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190_unique?,var_191_unique?,var_192_unique?,var_193_unique?,var_194_unique?,var_195_unique?,var_196_unique?,var_197_unique?,var_198_unique?,var_199_unique?
0,train_0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,0,0,0,0,0,0,0,0,0,0
1,train_1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,0,0,0,0,0,0,0,0,0,0
2,train_2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,0,0,0,0,0,0,0,0,0,0
3,train_3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,0,0,0,0,0,0,0,0,0,0
4,train_4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,0,0,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,train_199995,11.4880,-0.4956,8.2622,3.5142,10.3404,11.6081,5.6709,15.1516,-0.6209,...,0,1,0,0,0,0,1,0,0,0
199996,train_199996,4.9149,-2.4484,16.7052,6.6345,8.3096,-10.5628,5.8802,21.5940,-3.6797,...,0,0,0,1,0,0,0,0,0,0
199997,train_199997,11.2232,-5.0518,10.5127,5.6456,9.3410,-5.4086,4.5555,21.5571,0.1202,...,0,0,0,0,0,0,0,0,0,1
199998,train_199998,9.7148,-8.6098,13.6104,5.7930,12.5173,0.5339,6.0479,17.0152,-2.1926,...,0,0,0,0,0,0,0,0,0,1


Resampling

In [ ]:
training_set=x_train[x_train.columns[1:]].to_numpy()
training_labels=train['target'].to_numpy()

"""since we are dealing with an imbalanced data set we are going to deal with it  by oversampling the underrepresented class"""

from imblearn.over_sampling import RandomOverSampler
rov=RandomOverSampler()
x_resampled, y_resampled = rov.fit_resample(X=training_set, y=training_labels)

print('_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._')
print('training set shape before resampling: '+str(training_set.shape))
print('training set shape after: '+str(x_resampled.shape))


print('_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._')
print('training labels shape before resampling: '+str(training_labels.shape))
print('training labels shape after :'+str(y_resampled.shape))
print('_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._')

In [ ]:



"""initializes a histogram gradient boost classifier """
def get_hgbc():
    hgbc=sklearn.ensemble.HistGradientBoostingClassifier(

        #parameters
        max_iter=1000,
        validation_fraction=0.02,
        max_depth=None,
        learning_rate=0.1,
        loss='binary_crossentropy',
        l2_regularization=0.01,
        scoring='roc_auc',
        max_leaf_nodes=40,
        verbose=1

    )
    return hgbc



"""Creates and fits model with the resampled data"""
model1=get_hgbc()
model1.fit(x_resampled, y_resampled, sample_weight=None)
print('Training done!')



"""Let's make predictions with the model"""
#real_samples=x_test[:100000] #real samples
#fake_samples=x_test[100000:] #fake samples
super_test_set=x_test[x_test.columns[2:]].to_numpy()
predictions=model1.predict(super_test_set)


#TEST
predictions_prob=model1.predict_proba(super_test_set)
predictions_prob=pd.DataFrame(predictions_prob)
######
predictions


"""Let's get ready for submission. Creating a DataFrame containing ID codes and targets"""
submission_df=pd.DataFrame()
submission_df['ID_code']=x_test['ID_code']

#TEST
#submission_df['target']=predictions THIS IS THE REAL DEAL
submission_df['target']=predictions_prob[1].values #TEST
########################

"""we are also creating a dummy columns to help us sort our data by ID code"""
submission_df['dummy_index']=submission_df['ID_code'].apply(lambda x: int(x[5:]))

submission_df

"""Let's sort our data and save it as a CSV file! We are done!"""
juan_submission=submission_df.sort_values(by='dummy_index')
juan_submission.drop(['dummy_index'],axis=1).to_csv('juan_submission_v2.csv', index=False)
final_submission=pd.read_csv(filepath_or_buffer='juan_submission_v2.csv')